In [44]:
import torch
from torch import nn,optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# Loading Data

In [45]:
force = False
training_data = datasets.MNIST('data',train=True,download=force,transform=ToTensor())
test_data = datasets.MNIST('data',train=False,transform=ToTensor())

In [46]:
batch_size = 128
train_dataloader = DataLoader(training_data,batch_size=batch_size)
test_dataloader = DataLoader(test_data,batch_size=batch_size)
first_data = lambda dataloader : next(iter(dataloader))

X,y = first_data(train_dataloader)

print(f"Shape of X [N, C, H, W]: {X.shape}")
print(f"Shape of y: {y.shape} {y.dtype}")

X,y = first_data(test_dataloader)
print(f"Shape of X [N, C, H, W]: {X.shape}")
print(f"Shape of y: {y.shape} {y.dtype}")

Shape of X [N, C, H, W]: torch.Size([128, 1, 28, 28])
Shape of y: torch.Size([128]) torch.int64
Shape of X [N, C, H, W]: torch.Size([128, 1, 28, 28])
Shape of y: torch.Size([128]) torch.int64


# Device for Training

In [47]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Model Development

In [49]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 12, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(12, 24, kernel_size=6, stride=2),
            nn.ReLU(),
            nn.Conv2d(24, 32, kernel_size=6, stride=2),
            nn.ReLU(),
        )
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(288, 200),
            nn.ReLU(),
            nn.Linear(200, 10),
            nn.LogSoftmax(),
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (conv): Sequential(
    (0): Conv2d(1, 12, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(12, 24, kernel_size=(6, 6), stride=(2, 2))
    (3): ReLU()
    (4): Conv2d(24, 32, kernel_size=(6, 6), stride=(2, 2))
    (5): ReLU()
  )
  (avg_pool): AdaptiveAvgPool2d(output_size=(6, 6))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=288, out_features=200, bias=True)
    (1): ReLU()
    (2): Linear(in_features=200, out_features=10, bias=True)
    (3): LogSoftmax(dim=None)
  )
)


# Optimization configuration

In [50]:
# optimization configurations
loss_fn = nn.CrossEntropyLoss()
epochs = 10
max_learning_rate = 0.01
optimizer = torch.optim.Adam(model.parameters(), lr=max_learning_rate)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer,
                                          max_lr = max_learning_rate,
                                          total_steps = 10 * 60000,)

# Develop Optimization Iteration

In [51]:
def train(dataloader, model, loss_fn, optimizer,scheduler):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        # Print Stats
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [52]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

# Run the iteration

In [53]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer,scheduler)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


C:\Users\Leong Teng Man\AppData\Roaming\Python\Python310\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


loss: 2.294779  [  128/60000]
loss: 0.490490  [12928/60000]
loss: 0.280527  [25728/60000]
loss: 0.269921  [38528/60000]
loss: 0.293745  [51328/60000]
Test Error: 
 Accuracy: 94.4%, Avg loss: 0.185026 

Epoch 2
-------------------------------
loss: 0.151586  [  128/60000]
loss: 0.130690  [12928/60000]
loss: 0.139997  [25728/60000]
loss: 0.096407  [38528/60000]
loss: 0.175873  [51328/60000]
Test Error: 
 Accuracy: 96.7%, Avg loss: 0.102437 

Epoch 3
-------------------------------
loss: 0.073090  [  128/60000]
loss: 0.076799  [12928/60000]
loss: 0.084256  [25728/60000]
loss: 0.069463  [38528/60000]
loss: 0.125156  [51328/60000]
Test Error: 
 Accuracy: 97.5%, Avg loss: 0.076815 

Epoch 4
-------------------------------
loss: 0.070059  [  128/60000]
loss: 0.070484  [12928/60000]
loss: 0.054282  [25728/60000]
loss: 0.055872  [38528/60000]
loss: 0.111222  [51328/60000]
Test Error: 
 Accuracy: 97.6%, Avg loss: 0.069058 

Epoch 5
-------------------------------
loss: 0.057975  [  128/60000]
lo

# Save and Load Model

In [54]:
torch.save(model.state_dict(), "model/mnist_cnn.pth")
print("Saved PyTorch Model State to model/mnist_cnn.pth")

Saved PyTorch Model State to model/mnist_cnn.pth


In [55]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model/mnist_cnn.pth"))
model.to(device);

In [56]:
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 98.6%, Avg loss: 0.042701 



In [74]:
X = test_data.data.view(-1,1,28,28).float().to(device)
pred = torch.argmax(model(X),1)
res = pred.to('cpu')  == test_data.targets
sum(res)/len(res)

tensor(0.9810)